# Illustrations

In [270]:
from dhlab.module_update import update, css
update('nbpictures')
import json
import requests
import dhlab.nbpictures as nbp
import dhlab.nbtext as nb
from PIL import Image
from IPython.display import HTML
import re
%matplotlib inline

Updated file `C:\Users\larsj\Documents\GitHub\illustrations_bokhylla\nbpictures.py`

In [88]:
def get_illustrations(urn):
    r = requests.get('https://api.nb.no/ngram/illustrations', json={'urn': urn})
    return r.json() 

## Kode for å finne bilder og manipulere dem

In [274]:
small_scale = 0.59
large_scale = 1.58

def iiif_manifest(urn):
    r = requests.get("https://api.nb.no/catalog/v1/iiif/{urn}/manifest".format(urn=urn))
    return r.json()


    
def load_picture(url):
    r = requests.get(url, stream=True)
    r.raw.decode_content=True
    #print(r.status_code)
    return r.raw

def metadata(ill):
    #print(ill)
    u = bookurn_from_illustration(get_url_from_illustration(ill, part=100))
    m = iiif_manifest(u)['metadata']
    return m

def meta2html(ill):
    d = metadata(ill)
    ls = ""
    for i in d:
        ls += "<dt>{label}</dt><dd>{value}</dd>".format(label=i['label'], value=i['value'])
    return "<dl>{ls}</dl>".format(ls=ls)

def bookurn_from_illustration(ill):
    import re
    #print(ill)
    hits = re.findall("URN[^/]+", ill)
    if hits != []:
        res = hits[0].split('_')[:-1]
        res = '_'.join(res)
    return res

def get_url_from_illustration(illus, part = 200, scale = small_scale):
    urn = "URN:NBN:no-nb_" + illus['page']
    #print(urn)
    if part == True:
        # return cut out
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(int(illus['width'])*scale), 
            height = int(int(illus['height'])*scale), 
            vpos = int(int(illus['vpos'])*scale), 
            hpos = int(int(illus['hpos'])*scale)
        )
    else:
        # return whole page
        url = "https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg".format( part=part,
                urn = urn, width=illus['width'], height= illus['height'], vpos=illus['vpos'], hpos=illus['hpos'])
    
    return url

def get_url(illus, part = True, scale = None):
    """part sets size of output of page, if part is True it returns the cut out of image
    illus is a dictionary of with entries and values like this: 
    {'height': 270, 'hpos': 251, 'page': 'digibok_2017081626006_0018', 'resolution': 400, 'vpos': 791, 'width': 373} """
    
    if scale == None:
        if illus['resolution'] >= 300 or illus['resolution'] < 100:
            scale = large_scale
        else:
            scale = small_scale
    #scale = int(illus['resolution'])/250
    urn = "URN:NBN:no-nb_" + illus['page']
    #print(urn)
    if part == True:
        # return cut out
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(int(illus['width'])*scale), 
            height = int(int(illus['height'])*scale), 
            vpos = int(int(illus['vpos'])*scale), 
            hpos = int(int(illus['hpos'])*scale)
        )
    else:
        # return whole page
        url = "https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg".format( part=part,
                urn = urn, width=illus['width'], height= illus['height'], vpos=illus['vpos'], hpos=illus['hpos'])
    
    return url

def get_picture_from_illustration(illus, page=True, scale=small_scale):
    if page == True:
        url = get_url_from_illustration(illus, part=500)
    else:
        url = get_url_from_illustration(illus, part = True, scale=scale)
    return Image.open(load_picture(url))

def get_picture(illus, page=True, scale = small_scale):
    #if int(illus['resolution']) >= 400:
    #    scale = large_scale
    scale = int(illust['resolution'])/250
    if page == True:
        url = get_url_from_illustration(illus, part=500)
    else:
        url = get_url_from_illustration(illus, part = True, scale=scale)
    return Image.open(load_picture(url))

def display_finds(r):
    """A list of urls in r is displayed as HTML"""
    rows = ["<tr><td><img src='{row}'</td><td><a href = {meta} target='_'>{meta}</a></td></tr>".format(row=row, meta=row) for row in r]
    return HTML("""<html><head></head>
     <body>
     <table>
     {rows}
     </table>
     </body>
     </html>
     """.format(rows=' '.join(rows)))

def url2urn(url):
    return re.findall("URN:.*[0-9]{13}", url)[0]

In [275]:
url2urn("https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0018/396,1249,589,426/full/0/native.jpg")

'URN:NBN:no-nb_digibok_2017081626006'

In [262]:
bok = nb.best_book_urn(word = "knerten")
bok

[[2007050804047, 'Vestly, Anne-Cath.', 'Trofaste Knerten', 1963],
 [2007051001041, 'Andrew, Prudence', "Bråk i 10'ern", 1967],
 [2014081908070,
  'Lagerlöf, Selma',
  'Nils Holgerssons forunderlige reise',
  1927],
 [2010111605044,
  'Lagerlöf, Selma',
  'Nils Holgerssons forunderlige reise',
  1955],
 [2008081400066,
  'Lagerlöf, Selma',
  'Nils Holgerssons forunderlige reise',
  1998],
 [2008052104045,
  'Lagerlöf, Selma',
  'Nils Holgerssons forunderlige reise',
  1970],
 [2008031304011,
  'Lagerlöf, Selma',
  'Nils Holgerssons forunderlige reise',
  1995],
 [2007050804065, 'Vestly, Anne-Cath.', 'Lillebror og Knerten', 1962],
 [2007042001072,
  'Lagerlöf, Selma',
  'Nils Holgerssons forunderlige reise',
  1958],
 [2007032901041,
  'Lagerlöf, Selma',
  'Nils Holgerssons forunderlige reise',
  1963],
 [2007042401132, 'Andrew, Prudence', 'Ginger-gjengen gir seg ikke', 1966],
 [2007032101083, 'Andrew, Prudence', 'I dekning bak muren', 1963],
 [2006111000026, 'Neeraas, Synnøve Abusdal', 

In [269]:
illus = get_illustrations(2017081626006)
urls = [get_url(ill, part = True) for ill in illus]
display_finds(urls)

"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0018/396,1249,589,426/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0018/396,1249,589,426/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0019/543,755,265,504/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0019/543,755,265,504/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0020/379,589,507,440/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0020/379,589,507,440/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0021/180,237,959,518/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0021/180,237,959,518/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0022/274,510,755,372/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0022/274,510,755,372/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0023/372,401,480,590/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0023/372,401,480,590/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0025/357,797,627,704/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0025/357,797,627,704/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0026/257,481,780,594/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0026/257,481,780,594/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0027/252,254,807,535/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0027/252,254,807,535/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0027/292,1113,782,583/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0027/292,1113,782,583/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0028/216,578,878,649/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0028/216,578,878,649/full/0/native.jpg"


In [277]:
illus = get_illustrations(2009060904047
)
urls = [get_url(ill, part = 500) for ill in illus]
display_finds(urls)

"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0001/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0001/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0004/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0004/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0005/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0005/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0007/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0007/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0008/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0008/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0009/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0009/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0010/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0010/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0011/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0011/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0012/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0012/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0013/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0013/full/0,500/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0014/full/0,500/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2009060904047_0014/full/0,500/0/native.jpg"


In [230]:
illus

[{'height': 1921,
  'hpos': 0,
  'page': 'digibok_2016011207535_0002',
  'resolution': 400,
  'vpos': 0,
  'width': 1852},
 {'height': 1513,
  'hpos': 0,
  'page': 'digibok_2016011207535_0006',
  'resolution': 400,
  'vpos': 0,
  'width': 915},
 {'height': 923,
  'hpos': 933,
  'page': 'digibok_2016011207535_0009',
  'resolution': 400,
  'vpos': 997,
  'width': 919},
 {'height': 474,
  'hpos': 1276,
  'page': 'digibok_2016011207535_0011',
  'resolution': 400,
  'vpos': 469,
  'width': 179},
 {'height': 599,
  'hpos': 0,
  'page': 'digibok_2016011207535_0012',
  'resolution': 400,
  'vpos': 1039,
  'width': 1852},
 {'height': 194,
  'hpos': 290,
  'page': 'digibok_2016011207535_0012',
  'resolution': 400,
  'vpos': 1727,
  'width': 600},
 {'height': 1218,
  'hpos': 930,
  'page': 'digibok_2016011207535_0016',
  'resolution': 400,
  'vpos': 51,
  'width': 922},
 {'height': 220,
  'hpos': 923,
  'page': 'digibok_2016011207535_0020',
  'resolution': 400,
  'vpos': 1524,
  'width': 534},
 {